In [ ]:
#@title Setup model

import os
os.chdir('/content')

!unzip stylegan2.zip

!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

In [ ]:
#@title import modules

import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

%pip install -q ipywidgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from stylegan2.model import Generator

In [ ]:
#download checkpoint
#@title Setup files downloader
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

download_with_pydrive = True 

CODE_DIR = ''
class Downloader(object):
    def __init__(self, use_pydrive):
        self.use_pydrive = use_pydrive
        current_directory = os.getcwd()
        self.save_dir = os.path.join(os.path.dirname(current_directory), CODE_DIR, "pretrained_models")
        os.makedirs(self.save_dir, exist_ok=True)
        print(self.save_dir)
        if self.use_pydrive:
            self.authenticate()

    def authenticate(self):
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        self.drive = GoogleDrive(gauth)

    def download_file(self, file_id, file_name):
        file_dst = f'{self.save_dir}/{file_name}'
        if os.path.exists(file_dst):
            print(f'{file_name} already exists!')
            return
        if self.use_pydrive:
            downloaded = self.drive.CreateFile({'id':file_id})
            downloaded.FetchMetadata(fetch_all=True)
            downloaded.GetContentFile(file_dst)
        else:
            !gdown --id $file_id -O $file_dst

downloader = Downloader(download_with_pydrive)
id = '1EM87UquaoQmk17Q8d5kYIAHqu0dkYqdT'
file_name = 'stylegan2-ffhq-config-f.pt'

downloader.download_file(id, file_name)


/pretrained_models


In [ ]:
class Generator_Wrapper:
  def __init__(self):
    self.model = Generator(1024, 512, 8).to('cuda')
    self.init_weights()
    self.face_pool = torch.nn.AdaptiveAvgPool2d((256, 256))
  def init_weights(self):
    self.model.load_state_dict(torch.load('../pretrained_models/stylegan2-ffhq-config-f.pt')['g_ema'])

  def generate_image(self, input = None, input_is_latent = False, num_images = 1):
    if input is None:
      input = torch.randn((num_images, 512)).cuda()
    
    with torch.no_grad():
      out, latents = self.model([input], input_is_latent = input_is_latent, return_latents = True)
      out = self.face_pool(out)
      return out, latents

  def tensor2image(self, out):
    #TODO: given tensor of size (B, 3, 256, 256) in the range(-1, 1) return a PIL Image with size (256, 256 * B, 3)
    out = out.permute(0, 2, 3, 1).clamp(-1, 1).detach().cpu().numpy()
    out = ((out + 1) * 127.5).astype('uint8')

    out = np.concatenate(out, axis = 1)
    return Image.fromarray(out)
    

In [ ]:
G = Generator_Wrapper()
images, latents = G.generate_image(num_images = 4)

In [ ]:
G.tensor2image(images)

In [ ]:
#@title Latent interpolation

#TODO: generate two images with latents and visualize
images, latents = G.generate_image(num_images = 2)
G.tensor2image(images)

In [ ]:
#interpolate two latents
def mix_latents(mix_scale):
  #TODO: interpolate between two latents generated above. mix_scale is the interpolation coefficient
  new_latent = latents[0] * (mix_scale) + latents[1] * (1 - mix_scale)
  image, _ = G.generate_image(new_latent, input_is_latent = True)
  image = G.tensor2image(image)
  return image

In [ ]:
#make slider and show

interact(mix_latents, mix_scale=widgets.FloatSlider(0.5, min=0, max=1., step=0.05))